In [33]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
#import plotly.express as px
from xgboost import XGBClassifier
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, RandomizedSearchCV, GridSearchCV, train_test_split, KFold
import mlflow
import mlflow.sklearn


In [34]:

TRACKING_URL = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_URL)

In [35]:

loaded_model = mlflow.sklearn.load_model("./my_model")

In [36]:
source_data= "data/Student_performance_data _.csv"

def data_load(source_file,unique_maxcount):
    #read the file to dataframe
    df = pd.read_csv(source_file)

    # Distinction is based on the number of different values in the column
    columns = list(df.columns)
    categoric_columns = []
    numeric_columns = []
    for i in columns:
        if len(df[i].unique()) > unique_maxcount:
            numeric_columns.append(i)   
        else:
            categoric_columns.append(i)
    # Assuming the first column is an ID or non-numeric feature
    numeric_columns = numeric_columns[1:]

    # Convert numeric columns to float64
    df[numeric_columns] = df[numeric_columns].astype('float64')

    return df

def features(df):
    # CHOOSE THE TARGET FEATURE HERE, IN THIS CASE IT IS 'GradeClass'
    X = df.drop(columns=['GradeClass', 'GPA', 'StudentID', 'Age'])
    y = df['GradeClass']

    # Splitting the data into training and testing sets (e.g., 80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    return X_train, X_test, y_train, y_test


In [37]:
X_train, X_test, y_train, y_test = features(data_load(source_data, 5))
loaded_model.fit(X_train, y_train)
y_pred =loaded_model.predict(X_train)

In [38]:
y_pred

array([3., 4., 4., ..., 4., 4., 3.])

In [39]:
y_train

653     3.0
200     4.0
1606    4.0
1032    4.0
1293    4.0
       ... 
410     3.0
2115    2.0
719     4.0
2221    1.0
1553    3.0
Name: GradeClass, Length: 1913, dtype: float64